In [1]:
using LinearAlgebra
import Plots
import Roots # Root finding, I am using root_find
import Random
import StatsBase

In [15]:
curvyls = Vector{Function}(undef, 2) # Jump super operators
Ls = [ones(Float64, 2,2), zeros(Float64, 2,2)]
for k in 1:(size(Ls)[1])
    curvyls[k] = x -> x^2 #rho -> Ls[k]*rho*adjoint(Ls[k])
end 
      

In [4]:
# The System is defined by the jump operators and the Hamiltonian
struct System
    NLEVELS::Int64 #Number of levels of the system
    H::Matrix{ComplexF64} # Hamiltonian
    Ls::Vector{Matrix{ComplexF64}} # List of jump operators
    J::Matrix{ComplexF64} # Sum of Jump operators
    Heff::Matrix{ComplexF64} # Effective Hamiltonian
    ExpHeff::Function # Function that calculates the exponential of Heff at tau
    JumpSuperOperators::Vector{Function}  # Jump super operators
    NoJumpSuperOperator::Function # No-Jump superoperator
    # Inner Constructor
    function System(H::Matrix{ComplexF64}, Ls::Vector{Matrix{ComplexF64}})
        NLEVELS = size(H)[1] 
        NCHANNELS = size(Ls)[1] # Number of jump channels
        J = zeros(ComplexF64, NLEVELS, NLEVELS) 
        for L in Ls
            J = J + adjoint(L)*L
        end 
        CurvyLs = Vector{Function}(undef, NCHANNELS)
        for k in 1:NCHANNELS
            CurvyLs[k] = rho::Matrix{ComplexF64} -> Ls[k]*rho*adjoint(Ls[k])
        end 
        He = H - 0.5im*J
        expHe(tau::Float64) =  expm(-1im*tau*He)
        function expcurvyL0(rho::Matrix{ComplexF64}, tau::Float64)
            A = expHe(tau)
            return A*rho*adjoint(A)
        end 
        new(NLEVELS, H, Ls, J, He, expHe, CurvyLs,  expcurvyL0 )
    end 
end 

In [5]:
gamma = 1
sigma_m = [[0.0+0im, 0]  [1, 0]]
sys = System(
    zeros(ComplexF64, 2, 2), # Hamiltonian
    [sqrt(gamma)*sigma_m ] # Jump operators
)

System(2, ComplexF64[0.0 + 0.0im 0.0 + 0.0im; 0.0 + 0.0im 0.0 + 0.0im], Matrix{ComplexF64}[[0.0 + 0.0im 1.0 + 0.0im; 0.0 + 0.0im 0.0 + 0.0im]], ComplexF64[0.0 + 0.0im 0.0 + 0.0im; 0.0 + 0.0im 1.0 + 0.0im], ComplexF64[0.0 + 0.0im 0.0 + 0.0im; 0.0 + 0.0im 0.0 - 0.5im], var"#expHe#7"{Matrix{ComplexF64}}(ComplexF64[0.0 + 0.0im 0.0 + 0.0im; 0.0 + 0.0im 0.0 - 0.5im]), Function[var"#5#6"{Vector{Matrix{ComplexF64}}, Int64}(Matrix{ComplexF64}[[0.0 + 0.0im 1.0 + 0.0im; 0.0 + 0.0im 0.0 + 0.0im]], 1)], var"#expcurvyL0#8"{var"#expHe#7"{Matrix{ComplexF64}}}(var"#expHe#7"{Matrix{ComplexF64}}(ComplexF64[0.0 + 0.0im 0.0 + 0.0im; 0.0 + 0.0im 0.0 - 0.5im])))

In [20]:
gamma = 1
sys = System(2, [sqrt(gamma)*sigma_m])

System(2, Matrix{ComplexF64}[[0.0 + 0.0im 1.0 + 0.0im; 0.0 + 0.0im 0.0 + 0.0im]])

In [421]:
################### Setup ###############################
#1. Simulation conditions
NSAMPLES = 10000 # Number of points in the fine-grid, for the precomputation
seed = 1 # Seed, clearly
W = zeros(NSAMPLES) # To store the weights in sampling
const Trajectory = Vector{Tuple{Float64, Vector{Float64}}} # Alias for trajectory 
observed_trajectories = Vector{Trajectory}() # Vector with the trajectories
EPS = 1e-3
NTRAJ = 50 # Number of trajectories
tf = 1 # Final time
# Paramters
gamma = 1
# Initial State
psi0 = zeros(2)
psi0[2] = 1 # Initial condition
NLEVELS = size(psi0)[1]
# Jump Operators and related
sigma_m = [[0.0+0im, 0]  [1, 0]]
L1 = sqrt(gamma) * sigma_m
J = adjoint(L1)*L1
H_eff = -0.5im * J
expH_eff(tau) = exp(-1im*H_eff*tau) # Exponential of -i*tau*H_eff

################### Precomputation ######################
# Following Radaelli's advice, I'll choose a fine grid of time 
# To precompute J
multiplier = 10 # Factor to multiply by tf and create the sample
ts = LinRange(0, multiplier*tf, NSAMPLES)
Qs = Vector{Matrix{ComplexF64}}(undef, NSAMPLES)
dt = multiplier*tf/NSAMPLES
for k in 1:NSAMPLES
    expm = expH_eff(ts[k]) 
    Qs[k] = expm * J * adjoint(expm)
end 

@time begin
for j in 1:NTRAJ
Random.seed!(seed+j)
############### Running the Trajectory #################
# 0. Set Initial condition
    trajectory = Vector{Tuple{Float64,Vector{ComplexF64}}}()
    psi = zeros(2)
    psi = psi0
    t = 0
    while t < tf
    # 1. Calculate the WTD for the state, these act as weights
        for k in 1:NSAMPLES
           W[k] = real(dot(conj.(psi), Qs[k]*psi)) 
        end 
        # 1.a We must verify if we got a dark state, that can be cheked by 
        # looking at the normalization of the QTD
        if abs(sum(W)*dt - 1) > EPS
            break
        end 
        # 2. Sample jump time
        tau = StatsBase.sample(ts, weights(W))
        t = tau + t
        if t>tf # If the next jump happens after tf, stop 
            break
        end 
        psi_tilde = expH_eff(tau) * L1*psi # State without normalization
        psi = psi_tilde / norm(psi_tilde)
        push!(trajectory, (t, psi))
    end 
    push!(observed_trajectories, trajectory)
end 
end 

  0.666617 seconds (9.61 M allocations: 302.479 MiB, 11.33% gc time)


In [ ]:
function SampleSingleTrajectory
end